In [32]:
import nbconvert
import nbformat
from IPython.display import display, Markdown
from IPython import get_ipython

In [23]:
lab = nbformat.read('COMPAS_lab.ipynb', as_version =4)

In [24]:
lab.cells[0].source

'# Bias in AI Systems \n\n\n\n\n## Why COMPAS?\n\n\nPropublica started the COMPAS Debate with the article [Machine Bias](#References).  With their article, they also released details of their methodology and their [data and code](https://github.com/propublica/compas-analysis).  This presents a real data set that can be used for research on how data is used in a criminal justice settingn without researchers having to perform their own requests for information, so it has been used and reused a lot of times. \n'

In [25]:
lab.cells[3]

{'cell_type': 'code',
 'execution_count': 2,
 'metadata': {'name': 'import'},
 'outputs': [],
 'source': "import numpy as np\nimport pandas as pd\nimport scipy\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nimport itertools\nfrom sklearn.metrics import roc_curve\nfrom utilities import *\n%matplotlib inline\nimport warnings\nwarnings.filterwarnings('ignore')"}

In [60]:

class Tutorial:
    '''
    '''
    
    def __init__(self,filename):
        self.tutorial = nbformat.read(filename, as_version =4)
        self.current = 0
        
    def next(self):
        self.show(self.current)
        self.current += 1
        
    def start(self):
        self.show(0)
        self.current += 1
        
    def show(self,n):
        cell = lab.cells[n]
        if cell.cell_type == 'markdown':
            display(Markdown(cell.source))
        if cell.cell_type =='code':
            with open ('cell' +str(n) +'.py','w') as f:
                f.write(cell.source)
            load_cmd = 'cell' + str(n)
            get_ipython().run_line_magic('load',load_cmd)
#         display(Markdown('`' + load_cmd +'`' ))

In [61]:
tut = Tutorial('COMPAS_lab.ipynb')

In [62]:
tut.start()

# Bias in AI Systems 




## Why COMPAS?


Propublica started the COMPAS Debate with the article [Machine Bias](#References).  With their article, they also released details of their methodology and their [data and code](https://github.com/propublica/compas-analysis).  This presents a real data set that can be used for research on how data is used in a criminal justice settingn without researchers having to perform their own requests for information, so it has been used and reused a lot of times. 


In [63]:
tut.next()

<!--name:caseqs-->
## A COMPAS Case study

Next, let's look at what COMPAS is before we look at the data. 

The COMPAS score comes from the results of a [137 item survey](compas-core-sample.pdf).  It is distributed with a long [Practitioner's guide](compas_guide.pdf) that describes how it was developed and validated including which criminal theories it relies on. The claim is that COMPAS predicts two-year recidivism.  It has an accuracy around 67%.

### Discussion Questions

Breifly skim those and discuss things you find with your group. Make notes in the shared document about the following questions:

1. If the survey is administered through a social worker vs the defendent answering independently how might that impact responses?
1. How does knowing that the COMPAS score comes from that survey impact your view of it?  What problems might you predict? 
1. Who might this survey privelege?

In [64]:
tut.next()

<!--name:datainfo  -->
## Propublica COMPAS Data

The dataset consists of COMPAS scores assigned to defendants over two years 2013-2014 in Broward County, Florida. These scores are determined by a proprietary algorithm designed to evaluate a persons recidivism risk - the likelihood that they will reoffend. Risk scoring algorithms are widely used by judges to inform their scentencing and bail decisions in the criminal justice system in the United States. The original ProPublica analysis identified a number of fairness concerns around the use of COMPAS scores, including that ''black defendants were nearly twice as likely to be misclassified as higher risk compared to their white counterparts.'' Please see the full article for further details.



Let's get started by importing the libraries we will use.

In [66]:
# %load cell3
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from sklearn.metrics import roc_curve
from utilities import *
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [67]:
tut.next()

## Data prep

First we import the COMPAS dataset from the propublica repo and store it in a Pandas [dataframe](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.html) object.

In [69]:
# %load cell5
df = pd.read_csv("https://github.com/propublica/compas-analysis/raw/master/compas-scores-two-years.csv", 
                 header=0).set_index('id')


In [71]:
df.head()

,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
id,,,,,,,,,,,,,,,,,,,,,
1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,0,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [73]:
# %load cell6
df.to_csv('compas.csv')

In [74]:
tut.next()

First, let's take a look at this dataset. We print the features, and then the first entries.

In [77]:
# %load cell8
print(list(df))
df.head()

['name', 'first', 'last', 'compas_screening_date', 'sex', 'dob', 'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score', 'juv_misd_count', 'juv_other_count', 'priors_count', 'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number', 'c_offense_date', 'c_arrest_date', 'c_days_from_compas', 'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number', 'r_charge_degree', 'r_days_from_arrest', 'r_offense_date', 'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid', 'is_violent_recid', 'vr_case_number', 'vr_charge_degree', 'vr_offense_date', 'vr_charge_desc', 'type_of_assessment', 'decile_score.1', 'score_text', 'screening_date', 'v_type_of_assessment', 'v_decile_score', 'v_score_text', 'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1', 'start', 'end', 'event', 'two_year_recid']


,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
id,,,,,,,,,,,,,,,,,,,,,
1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,0,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [76]:
tut.next()

### Data Cleaning

For this analysis, we will restrict ourselves to only a few features, and clean the dataset according to the methods using in the original ProPublica analysis. 

Details of the cleaning method can be found in the utilities file.

In [79]:
# %load cell10
# Select features that will be analyzed
features_to_keep = ['age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex', 'priors_count', 
                    'days_b_screening_arrest', 'decile_score', 'is_recid', 'two_year_recid', 'c_jail_in', 'c_jail_out']
df = df[features_to_keep]
df = clean_compas(df)
df.head()
print("\ndataset shape (rows, columns)", df.shape)

Number of rows removed: 1042
Features: ['age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex', 'priors_count', 'days_b_screening_arrest', 'decile_score', 'is_recid', 'two_year_recid', 'c_jail_in', 'c_jail_out', 'length_of_stay']

dataset shape (rows, columns) (6172, 14)


In [81]:
# %load cell11
df.to_csv('compas_clean.csv')

In [82]:
tut.next()

## Data Exploration

Next we provide a few ways to look at the relationships between the attributes in the dataset. Here is an explanation of these values:

* `age`: defendant's age
* `c_charge_degree`: degree charged (Misdemeanor of Felony)
* `race`: defendant's race
* `age_cat`: defendant's age quantized in "less than 25", "25-45", or "over 45"
* `score_text`: COMPAS score: 'low'(1 to 5), 'medium' (5 to 7), and 'high' (8 to 10).
* `sex`: defendant's gender
* `priors_count`: number of prior charges
* `days_b_screening_arrest`: number of days between charge date and arrest where defendant was screened for compas score
* `decile_score`: COMPAS score from 1 to 10 (low risk to high risk)
* `is_recid`: if the defendant recidivized
* `two_year_recid`: if the defendant within two years
* `c_jail_in`: date defendant was imprisoned
* `c_jail_out`: date defendant was released from jail
* `length_of_stay`: length of jail stay

In particular, as in the ProPublica analysis, we are interested in the implications for the treatment of different groups as defined by some **sensitive data attributes**. In particular we will consider race as the protected attribute in our analysis. Next we look at the number of entries for each race.


<font color=red> Another interesting fairness analysis might be to consider group outcomes by gender or age. In fact, a [2017 appeal to the US Suprme Court](https://en.wikipedia.org/wiki/Loomis_v._Wisconsin) challenged the role of gender in determining COMPAS scores.</font> 